In [47]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
)

import torch.nn as nn

def has_sentences(example):
    """
    Функция возвращает True, если пример содержит хотя бы одно предложение
    в documents_sentences (и, опционально, в response_sentences).
    Иначе возвращает False.
    """

    if "documents_sentences" not in example:
        return False
    if not isinstance(example["documents_sentences"], list):
        return False
    total_doc_sentences = 0
    for doc in example["documents_sentences"]:
        if isinstance(doc, list):
            total_doc_sentences += len(doc)
    if total_doc_sentences == 0:
        return False

    if "response_sentences" in example:
        if not isinstance(example["response_sentences"], list):
            return False
        if len(example["response_sentences"]) == 0:
            return False

    return True

MODEL_NAME = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def preprocess(example, max_length=1024):
    question_ids = tokenizer.encode(example["question"], add_special_tokens=False)
    doc_ids, rel_labels, util_labels = [], [], []

    for doc in example["documents_sentences"]:
        for key, sent in doc:
            tokens = tokenizer.encode(sent, add_special_tokens=False)
            doc_ids += tokens
            rel_labels += [float(key in example["all_relevant_sentence_keys"])] * len(tokens)
            util_labels += [float(key in example["all_utilized_sentence_keys"])] * len(tokens)

    response_ids = tokenizer.encode(example["response"], add_special_tokens=False)
    adh_labels = [float(example["adherence_score"])] * len(response_ids)

    sep_id = tokenizer.sep_token_id
    input_ids = question_ids + [sep_id] + doc_ids + [sep_id] + response_ids

    context_mask = [0] * (len(question_ids)+1) + [1]*len(doc_ids) + [0] + [0]*len(response_ids)
    response_mask = [0]*(len(question_ids)+len(doc_ids)+2) + [1]*len(response_ids)

    rel_labels = [0.0]*(len(question_ids)+1) + rel_labels + [0.0]*(len(response_ids)+1)
    util_labels = [0.0]*(len(question_ids)+1) + util_labels + [0.0]*(len(response_ids)+1)
    adh_labels = [0.0]*(len(question_ids)+len(doc_ids)+2) + adh_labels

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        context_mask = context_mask[:max_length]
        response_mask = response_mask[:max_length]
        rel_labels = rel_labels[:max_length]
        util_labels = util_labels[:max_length]
        adh_labels = adh_labels[:max_length]

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor([1]*len(input_ids), dtype=torch.long),
        'context_mask': torch.tensor(context_mask, dtype=torch.bool),
        'response_mask': torch.tensor(response_mask, dtype=torch.bool),
        'labels_relevance': torch.tensor(rel_labels, dtype=torch.float),
        'labels_utilization': torch.tensor(util_labels, dtype=torch.float),
        'labels_adherence': torch.tensor(adh_labels, dtype=torch.float),
    }





In [23]:
from promt_test.data import load_full_ragbench

In [24]:
dataset = load_full_ragbench()#load_dataset("rungalileo/ragbench", "delucionqa")
dataset = {key:dataset[key].filter(has_sentences) for key in dataset}

In [25]:
MAX_LEN = tokenizer.model_max_length  # максимально допустимая длина входа

In [26]:
def count_tokens(example):
    question_ids = tokenizer.encode(example["question"], add_special_tokens=False)
    doc_ids = []
    for doc in example["documents_sentences"]:
        for key, sent in doc:
            doc_ids.extend(tokenizer.encode(sent, add_special_tokens=False))
    response_ids = tokenizer.encode(example["response"], add_special_tokens=False)
    total = len(question_ids) + 1 + len(doc_ids) + 1 + len(response_ids)
    return total

def filter_by_length(example):
    return count_tokens(example) <= MAX_LEN


filtered = {
    ds_name: ds_dict.filter(filter_by_length)
    for ds_name, ds_dict in dataset.items()
}


Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1621/1621 [00:18<00:00, 88.90 examples/s]


In [27]:

train_splits = [ds["train"] for ds in filtered.values() if "train" in ds]
val_splits   = [ds["validation"] for ds in filtered.values() if "validation" in ds]
test_splits  = [ds["test"] for ds in filtered.values() if "test" in ds]

In [28]:
from datasets import DatasetDict, concatenate_datasets
combined = DatasetDict({
    "train":      concatenate_datasets(train_splits),
    "validation": concatenate_datasets(val_splits),
    "test":       concatenate_datasets(test_splits)
})
combined

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score'],
        num_rows: 26966
    })
    validation: Dataset({
        features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 're

In [29]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     # batch: list of dicts
#     keys = batch[0].keys()
#     out = {}
#     #  max len
#     max_len = MAX_LEN #max(len(x['input_ids']) for x in batch)
# #     print(batch)
#     for k in keys:
#         vals = [x[k] for x in batch]
#         if k == 'input_ids':
#             out[k] = pad_sequence(vals, batch_first=True, padding_value=tokenizer.pad_token_id)
#         elif k == 'attention_mask':
#             out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
#         elif k in ['context_mask', 'response_mask']:
#             out[k] = pad_sequence(vals, batch_first=True, padding_value=False)
#         elif k.startswith('labels_'):
#             out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
#         else:
#             out[k] = torch.stack(vals)
#     return out
# train_data = [preprocess(ex) for ex in combined['train']]
# val_data   = [preprocess(ex) for ex in combined['validation']]
# batch_size=8

# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [30]:
def pad_or_truncate(x: torch.Tensor, max_len: int, pad_value: int):
    L = x.size(0)
    if L < max_len:
        # дополняем в конец
        pad = x.new_full((max_len - L,), pad_value)
        return torch.cat([x, pad], dim=0)
    else:
        # усекаем
        return x[:max_len]

def collate_fn(batch):
    keys = batch[0].keys()
    out = {}
    max_len = MAX_LEN

    for k in keys:
        vals = [x[k] for x in batch]
        if k == 'input_ids':
            # сначала усекаем/падим каждый, потом стэкаем
            processed = [pad_or_truncate(x, max_len, tokenizer.pad_token_id) for x in vals]
            out[k] = torch.stack(processed, dim=0)
        elif k == 'attention_mask':
            processed = [pad_or_truncate(x, max_len, 0) for x in vals]
            out[k] = torch.stack(processed, dim=0)
        elif k in ['context_mask', 'response_mask']:
            processed = [pad_or_truncate(x.to(torch.uint8), max_len, 0).to(torch.bool) for x in vals]
            out[k] = torch.stack(processed, dim=0)
        elif k.startswith('labels_'):
            processed = [pad_or_truncate(x, max_len, 0) for x in vals]
            out[k] = torch.stack(processed, dim=0)
        else:
            out[k] = torch.stack(vals, dim=0)
    return out
train_data = [preprocess(ex) for ex in combined['train']]
val_data   = [preprocess(ex) for ex in combined['validation']]
batch_size=8

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [31]:
combined['train'][1].keys()

dict_keys(['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score'])

In [32]:
import numpy as np

In [33]:
np.mean(combined['validation']['relevance_score']), np.mean(combined['validation']['utilization_score']), np.mean(combined['validation']['completeness_score'])

(0.34462288803319874, 0.2855946234091305, 0.8451311705064115)

In [34]:
processed = preprocess(combined['train'][1])


tokens = tokenizer.convert_ids_to_tokens(processed["input_ids"])

labels_rel = processed["labels_relevance"]
labels_util = processed["labels_utilization"]
labels_adh = processed["labels_adherence"]

print(f"{'Token':20} | {'Relevance':9} | {'Utilization':11} | {'Adherence':9}")
print("-" * 60)
i = 0
for token, rel, util, adh in zip(tokens, labels_rel, labels_util, labels_adh):
    if token == '[PAD]':
        break
    i+=1
    print(f"{token:20} | {rel:9.1f} | {util:11.1f} | {adh:9.1f}")
    if token == '▁Simply':
        print(i)

Token                | Relevance | Utilization | Adherence
------------------------------------------------------------
▁What                |       0.0 |         0.0 |       0.0
▁CO                  |       0.0 |         0.0 |       0.0
V                    |       0.0 |         0.0 |       0.0
s                    |       0.0 |         0.0 |       0.0
▁were                |       0.0 |         0.0 |       0.0
▁known               |       0.0 |         0.0 |       0.0
▁to                  |       0.0 |         0.0 |       0.0
▁infect              |       0.0 |         0.0 |       0.0
▁humans              |       0.0 |         0.0 |       0.0
▁before              |       0.0 |         0.0 |       0.0
▁December            |       0.0 |         0.0 |       0.0
▁2019                |       0.0 |         0.0 |       0.0
?                    |       0.0 |         0.0 |       0.0
[SEP]                |       0.0 |         0.0 |       0.0
▁Title               |       0.0 |         0.0 |      

In [48]:
import torch.nn.functional as F
from scipy.special import expit
import numpy as np
class DebertaTrace(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        hid = base_model.config.hidden_size
        self.rel_head = nn.Linear(hid,1)
        self.util_head = nn.Linear(hid,1)
        self.adh_head = nn.Linear(hid,1)

    def forward(self, input_ids, attention_mask):
        out = self.base(input_ids=input_ids, attention_mask=attention_mask)
        hs = out.last_hidden_state
        return {
            'logits_relevance': self.rel_head(hs).squeeze(-1),
            'logits_utilization': self.util_head(hs).squeeze(-1),
            'logits_adherence': self.adh_head(hs).squeeze(-1)
        }


def compute_batch_metrics(logits, labels, masks):
    # logits: dict of tensors [B,L]
    # labels: dict of tensors [B,L]
    # masks: dict of bool tensors [B,L]
    rel_log, util_log, adh_log = logits['logits_relevance'], logits['logits_utilization'], logits['logits_adherence']
    rel_lab, util_lab, adh_lab = labels['labels_relevance'], labels['labels_utilization'], labels['labels_adherence']
    ctx_m, resp_m = masks['context_mask'], masks['response_mask']

    # probs
    rel_p = torch.tensor(expit(rel_log.detach().cpu().numpy()))
    util_p = torch.tensor(expit(util_log.detach().cpu().numpy()))
    adh_p = torch.tensor(expit(adh_log.detach().cpu().numpy()))
    # preds
    pred_rel = rel_p>0.5
    pred_util = util_p>0.5
    pred_adh = adh_p>0.5
    # true
    true_rel = rel_lab.detach().cpu()==1
    true_util = util_lab.detach().cpu()==1
    true_adh = adh_lab.detach().cpu()==1

    # per example
    def ex_metric(pred, true, mask):
        inter = (pred & true & mask).sum(dim=1).float()
        denom = (true & mask).sum(dim=1).float()
        return (inter/denom.clamp(min=1)).mean().item()

    rel_m = ex_metric(pred_rel, true_rel, ctx_m)
    util_m= ex_metric(pred_util, true_util, ctx_m)
    adh_m = ex_metric(pred_adh, true_adh, resp_m)
    comp = ((pred_rel & pred_util & ctx_m).sum(dim=1).float()/ (true_rel & ctx_m).sum(dim=1).float().clamp(min=1)).mean().item()
    return rel_m, util_m, adh_m, comp

In [36]:
def check_for_nans(logits, labels, mask, head_name=""):
    logit_vals = logits[mask]
    label_vals = labels[mask]
    
    nan_logits = torch.isnan(logit_vals)
    nan_labels = torch.isnan(label_vals)
    
    if nan_logits.any() or nan_labels.any():
        print(f"NaN detected in head '{head_name}':")
        if nan_logits.any():
            idxs = torch.nonzero(nan_logits, as_tuple=True)[0]
            print(f"  logits NaN at positions: {idxs.tolist()}")
            print(f"  sample logits: {logit_vals[idxs].tolist()}")
        if nan_labels.any():
            idxs = torch.nonzero(nan_labels, as_tuple=True)[0]
            print(f"  labels NaN at positions: {idxs.tolist()}")
            print(f"  sample labels: {label_vals[idxs].tolist()}")
        # Остановим обучение для отладки
        raise ValueError(f"NaN encountered in head '{head_name}'")

In [37]:
def compute_trace_metrics_inference(logits, masks, threshold=0.5):

    rel_pred  = (torch.sigmoid(logits['logits_relevance'].detach().cpu())  > threshold)
    util_pred = (torch.sigmoid(logits['logits_utilization'].detach().cpu())> threshold)
    adh_pred  = (torch.sigmoid(logits['logits_adherence'].detach().cpu())   > threshold)

    ctx_m  = masks['context_mask'].detach().cpu()
    resp_m = masks['response_mask'].detach().cpu()

    def rate(pred, mask):
        # sum(pred & mask) / sum(mask)
        num = (pred & mask).sum(dim=1).float()
        den = mask.sum(dim=1).float().clamp(min=1)
        return num.div(den)

    relevance_rate   = rate(rel_pred,  ctx_m)
    utilization_rate = rate(util_pred, ctx_m)
    adherence_rate   = rate(adh_pred,  resp_m)

    # completeness: из релевантных предсказаний — сколько ещё и util
    num_ru = (rel_pred & util_pred & ctx_m).sum(dim=1).float()
    den_r  = rel_pred.sum(dim=1).float().clamp(min=1)
    completeness = num_ru.div(den_r)

    return {
        'relevance_rate':   relevance_rate,    
        'utilization_rate': utilization_rate,  
        'adherence_rate':   adherence_rate,
        'completeness':     completeness
    }

In [38]:
def compute_classification_metrics(logits, labels, masks):
    rel_pred  = (torch.sigmoid(logits['logits_relevance'].detach().cpu())  > 0.5) & masks['context_mask']
    util_pred = (torch.sigmoid(logits['logits_utilization'].detach().cpu())> 0.5) & masks['context_mask']
    adh_pred  = (torch.sigmoid(logits['logits_adherence'].detach().cpu())   > 0.5) & masks['response_mask']

    rel_true  = (labels['labels_relevance']   == 1) & masks['context_mask']
    util_true = (labels['labels_utilization']== 1) & masks['context_mask']
    adh_true  = (labels['labels_adherence']   == 1) & masks['response_mask']

    def prf(pred, true):
        tp = (pred & true).sum().float()
        fp = (pred & ~true).sum().float()
        fn = (~pred & true).sum().float()
        prec = tp.div(tp + fp.clamp(min=1))
        rec  = tp.div(tp + fn.clamp(min=1))
        f1   = 2*prec*rec.div((prec+rec).clamp(min=1))
        return prec.item(), rec.item(), f1.item()

    rel_p,  rel_r,  rel_f1  = prf(rel_pred,  rel_true)
    util_p, util_r, util_f1 = prf(util_pred, util_true)
    adh_p,  adh_r,  adh_f1  = prf(adh_pred,  adh_true)

    return {
        'relevance_precision':    rel_p,
        'relevance_recall':       rel_r,
        'relevance_f1':           rel_f1,
        'utilization_precision':  util_p,
        'utilization_recall':     util_r,
        'utilization_f1':         util_f1,
        'adherence_precision':    adh_p,
        'adherence_recall':       adh_r,
        'adherence_f1':           adh_f1,
    }

In [39]:
val_metrics_per_examples = []

In [40]:
import torch

def compute_classification_metrics_stream(all_logits, all_labels, all_masks, threshold=0.5):
    rel_pred_list, rel_true_list = [], []
    util_pred_list, util_true_list = [], []
    adh_pred_list, adh_true_list   = [], []

    for (
        lr, lu, la,
        tr, tu, ta,
        ctx_m, resp_m
    ) in zip(
        all_logits['logits_relevance'],
        all_logits['logits_utilization'],
        all_logits['logits_adherence'],
        all_labels['labels_relevance'],
        all_labels['labels_utilization'],
        all_labels['labels_adherence'],
        all_masks['context_mask'],
        all_masks['response_mask'],
    ):
        p_rel  = (torch.sigmoid(lr) > threshold)
        p_util = (torch.sigmoid(lu) > threshold)
        p_adh  = (torch.sigmoid(la) > threshold)
        t_rel  = (tr == 1)
        t_util = (tu == 1)
        t_adh  = (ta == 1)

        rel_pred_list.append( p_rel[ctx_m].cpu() )
        rel_true_list.append( t_rel[ctx_m].cpu() )
        util_pred_list.append( p_util[ctx_m].cpu() )
        util_true_list.append( t_util[ctx_m].cpu() )
        adh_pred_list.append( p_adh[resp_m].cpu() )
        adh_true_list.append( t_adh[resp_m].cpu() )

    rel_pred_all  = torch.cat(rel_pred_list)
    rel_true_all  = torch.cat(rel_true_list)
    util_pred_all = torch.cat(util_pred_list)
    util_true_all = torch.cat(util_true_list)
    adh_pred_all  = torch.cat(adh_pred_list)
    adh_true_all  = torch.cat(adh_true_list)

    def prf(pred, true):
        tp = int((pred & true).sum())
        fp = int((pred & ~true).sum())
        fn = int((~pred & true).sum())
        prec = tp / (tp + fp) if tp + fp > 0 else 0.0
        rec  = tp / (tp + fn) if tp + fn > 0 else 0.0
        f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) > 0 else 0.0
        return prec, rec, f1

    rel_p, rel_r, rel_f1   = prf(rel_pred_all,  rel_true_all)
    util_p, util_r, util_f1 = prf(util_pred_all, util_true_all)
    adh_p, adh_r, adh_f1    = prf(adh_pred_all,  adh_true_all)

    return {
        'relevance_precision':    rel_p,
        'relevance_recall':       rel_r,
        'relevance_f1':           rel_f1,
        'utilization_precision':  util_p,
        'utilization_recall':     util_r,
        'utilization_f1':         util_f1,
        'adherence_precision':    adh_p,
        'adherence_recall':       adh_r,
        'adherence_f1':           adh_f1,
    }


In [41]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base = AutoModel.from_pretrained(MODEL_NAME)
model=DebertaTrace(base).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler = torch.amp.GradScaler('cuda')
accum=4

In [50]:
pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
!huggingface-cli login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 51, in main
    service.run()
  File "/usr/local/lib/python3

In [58]:
# base_model = AutoModel.from_pretrained(MODEL_NAME)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = DebertaTrace(base_model)

# model.load_state_dict(torch.load("../tmp/model_state_dict.pth", weights_only=True))

# base_model.save_pretrained("../model_repo")

# torch.save({
#     'rel_head': model.rel_head.state_dict(),
#     'util_head': model.util_head.state_dict(),
#     'adh_head': model.adh_head.state_dict()
# }, "../model_repo/heads_weights.pt")
# tokenizer.save_pretrained("../model_repo")

('../model_repo/tokenizer_config.json',
 '../model_repo/special_tokens_map.json',
 '../model_repo/spm.model',
 '../model_repo/added_tokens.json',
 '../model_repo/tokenizer.json')

In [59]:
# from huggingface_hub import HfApi

# 
# api = HfApi()

# folder_path = "../model_repo"

# repo_id = "CMCenjoyer/deberta-trace"
# api.create_repo(repo_id, repo_type="model", exist_ok=True)
# api.upload_folder(
#     folder_path=folder_path,
#     repo_id=repo_id,
#     repo_type="model",
#     token=True
# )

spm.model: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.46M/2.46M [00:00<00:00, 3.28MB/s]


CommitInfo(commit_url='https://huggingface.co/CMCenjoyer/deberta-trace/commit/d77afe2c560a6573722a2f662237ff319b063c69', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d77afe2c560a6573722a2f662237ff319b063c69', pr_url=None, pr_revision=None, pr_num=None)

In [21]:


for epoch in range(5):
    model.train()
    total_loss=0
    opt.zero_grad()
    loop = tqdm(train_loader, desc=f"Training Epoch {epoch}", total=len(train_loader))
    for i, batch in enumerate(loop, 1):
        inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
        labels = {k: v.to(device) for k,v in batch.items() if k.startswith('labels_')}
        masks  = {k: v.to(device) for k,v in batch.items() if k.endswith('_mask')}
        with torch.amp.autocast('cuda'):
            out = model(**inputs)
            
            l_rel = F.binary_cross_entropy_with_logits(out['logits_relevance'][masks['context_mask']], labels['labels_relevance'][masks['context_mask']])

            l_util= F.binary_cross_entropy_with_logits(out['logits_utilization'][masks['context_mask']], labels['labels_utilization'][masks['context_mask']])

            l_adh = F.binary_cross_entropy_with_logits(out['logits_adherence'][masks['response_mask']], labels['labels_adherence'][masks['response_mask']])
            loss = (l_rel + l_util + l_adh)/ 3 #/ accum
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         if (i+1)%accum==0:
        scaler.step(opt)
        scaler.update()
        opt.zero_grad()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
#         break
    print(f"Epoch {epoch} loss: {total_loss:.4f}")

    # валидация
    model.eval()
    sums = np.zeros(4);
    count=0
    metrics = {
    'relevance_rate':   [],  
    'utilization_rate': [],
    'adherence_rate':   [],
    'completeness':     []
    }
    all_logits = {
        'logits_relevance':   [],
        'logits_utilization': [],
        'logits_adherence':   []
    }
    all_labels = {
        'labels_relevance':   [],
        'labels_utilization': [],
        'labels_adherence':   []
    }
    all_masks = {
        'context_mask':  [],
        'response_mask': []
    }
    with torch.no_grad():
        
        for batch in tqdm(val_loader):
            inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
            labels = {k: v for k,v in batch.items() if k.startswith('labels_')}
            masks  = {k: v for k,v in batch.items() if k.endswith('_mask')}
            out = model(**{k:inputs[k] for k in inputs})
            batch_metrics = compute_trace_metrics_inference(out, masks)
#             m = compute_batch_metrics(out, labels, masks)
            for name, values in batch_metrics.items():
                metrics[name].extend(values.cpu().tolist())
#             sums += np.array(m)
            for k in all_logits:
#                 print(out[k].shape)
                all_logits[k].append(out[k].cpu())
            for k in all_labels:
                all_labels[k].append(labels[k].cpu())
            for k in all_masks:
                all_masks[k].append(masks[k].cpu())
    
    big_logits = { k: torch.cat(v, dim=0) for k, v in all_logits.items() }
    big_labels = { k: torch.cat(v, dim=0) for k, v in all_labels.items() }
    big_masks  = { k: torch.cat(v, dim=0) for k, v in all_masks.items() }
    class_metrics = compute_classification_metrics(big_logits, big_labels, big_masks)
    print("\nClassification metrics on full validation set:")
    for name, val in class_metrics.items():
        print(f"  {name}: {val:.4f}")
#     avg = sums/count
    print(f"Eval -> relevance={np.mean(metrics['relevance_rate']):.4f}, utilization={np.mean(metrics['utilization_rate']):.4f}, adherence={np.mean(metrics['adherence_rate']):.4f}, completeness={np.mean(metrics['completeness']):.4f}")


Training Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3371/3371 [15:41<00:00,  3.58it/s, loss=0.326]


Epoch 0 loss: 1063.1702


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [00:47<00:00,  9.63it/s]



Classification metrics on full validation set:
  relevance_precision: 0.6637
  relevance_recall: 0.9367
  relevance_f1: 0.7769
  utilization_precision: 0.7290
  utilization_recall: 0.9272
  utilization_f1: 0.8162
  adherence_precision: 0.7859
  adherence_recall: 0.9994
  adherence_f1: 0.8799
Eval -> relevance=0.5682, utilization=0.4437, adherence=0.9970, completeness=0.3126


Training Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3371/3371 [15:42<00:00,  3.58it/s, loss=0.265]


...


Classification metrics on full validation set:
  relevance_precision: 0.7031
  relevance_recall: 0.9177
  relevance_f1: 0.7962
  utilization_precision: 0.7402
  utilization_recall: 0.9224
  utilization_f1: 0.8213
  adherence_precision: 0.8987
  adherence_recall: 0.8805
  adherence_f1: 0.8895
Eval -> relevance=0.5241, utilization=0.4332, adherence=0.8508, completeness=0.3147


Training Epoch 4: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3371/3371 [15:41<00:00,  3.58it/s, loss=0.0737]


Epoch 4 loss: 444.0855


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [00:47<00:00,  9.65it/s]



Classification metrics on full validation set:
  relevance_precision: 0.7078
  relevance_recall: 0.9131
  relevance_f1: 0.7975
  utilization_precision: 0.7448
  utilization_recall: 0.9291
  utilization_f1: 0.8268
  adherence_precision: 0.8770
  adherence_recall: 0.9249
  adherence_f1: 0.9003
Eval -> relevance=0.5170, utilization=0.4333, adherence=0.8913, completeness=0.3332


In [ ]:
Classification metrics on full validation set:
  relevance_precision: 0.7078
  relevance_recall: 0.9131
  relevance_f1: 0.7975
  utilization_precision: 0.7448
  utilization_recall: 0.9291
  utilization_f1: 0.8268
  adherence_precision: 0.8770
  adherence_recall: 0.9249
  adherence_f1: 0.9003
Eval -> relevance=0.5170, utilization=0.4333, adherence=0.8913, completeness=0.3332

In [26]:

# torch.save(model.state_dict(), '../tmp/model_state_dict.pth')

In [43]:
model.load_state_dict(torch.load('../tmp/model_state_dict.pth'))

/tmp/ipykernel_8672/777383360.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../tmp/model_state_dict.pth'))


<All keys matched successfully>

In [44]:
metrics = {
'relevance_rate':   [],  
'utilization_rate': [],
'adherence_rate':   [],
'completeness':     []
}
all_logits = {
    'logits_relevance':   [],
    'logits_utilization': [],
    'logits_adherence':   []
}
all_labels = {
    'labels_relevance':   [],
    'labels_utilization': [],
    'labels_adherence':   []
}
all_masks = {
    'context_mask':  [],
    'response_mask': []
}
with torch.no_grad():

    for batch in tqdm(val_loader):
        inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
        labels = {k: v for k,v in batch.items() if k.startswith('labels_')}
        masks  = {k: v for k,v in batch.items() if k.endswith('_mask')}
        out = model(**{k:inputs[k] for k in inputs})
        batch_metrics = compute_trace_metrics_inference(out, masks)
#             m = compute_batch_metrics(out, labels, masks)
        for name, values in batch_metrics.items():
            metrics[name].extend(values.cpu().tolist())
#             sums += np.array(m)
        for k in all_logits:
#                 print(out[k].shape)
            all_logits[k].append(out[k].cpu())
        for k in all_labels:
            all_labels[k].append(labels[k].cpu())
        for k in all_masks:
            all_masks[k].append(masks[k].cpu())
    big_logits = { k: torch.cat(v, dim=0) for k, v in all_logits.items() }
    big_labels = { k: torch.cat(v, dim=0) for k, v in all_labels.items() }
    big_masks  = { k: torch.cat(v, dim=0) for k, v in all_masks.items() }
    class_metrics = compute_classification_metrics(big_logits, big_labels, big_masks)
    print("\nClassification metrics on full validation set:")
    for name, val in class_metrics.items():
        print(f"  {name}: {val:.4f}")
#     avg = sums/count
    print(f"Eval -> relevance={np.mean(metrics['relevance_rate']):.4f}, utilization={np.mean(metrics['utilization_rate']):.4f}, adherence={np.mean(metrics['adherence_rate']):.4f}, completeness={np.mean(metrics['completeness']):.4f}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [00:47<00:00,  9.63it/s]



Classification metrics on full validation set:
  relevance_precision: 0.6925
  relevance_recall: 0.9221
  relevance_f1: 0.7910
  utilization_precision: 0.7372
  utilization_recall: 0.9340
  utilization_f1: 0.8240
  adherence_precision: 0.8590
  adherence_recall: 0.9343
  adherence_f1: 0.8951
Eval -> relevance=0.5330, utilization=0.4406, adherence=0.9020, completeness=0.3665


In [45]:
from scipy import stats

r, p_value = stats.pearsonr(combined['validation']['relevance_score'], metrics['relevance_rate'])
print("Коэффициент корреляции Пирсона:", r)
print("p-value:", p_value)

Коэффициент корреляции Пирсона: 0.45373879733665534
p-value: 6.445192852460025e-184


In [46]:
r, p_value = stats.pearsonr(combined['validation']['utilization_score'], metrics['utilization_rate'])
print("Коэффициент корреляции Пирсона:", r)
print("p-value:", p_value)

Коэффициент корреляции Пирсона: 0.552252854067706
p-value: 4.7870300467656045e-289
